# CONFLUENCE Tutorial: Elevation-Based HRU Discretization

This notebook demonstrates elevation-based HRU discretization, building on the distributed domain from Tutorial 3. We'll:

1. Use the existing GRUs from Tutorial 3
2. Apply elevation-based discretization
3. Run the model
4. Compare results with lumped and GRU-based approaches

**Prerequisites**: Tutorial 3 must be completed successfully.

## 1. Setup and Import Libraries

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import numpy as np
import contextily as cx
import xarray as xr
import shutil
from IPython.display import Image, display

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import main CONFLUENCE class
from CONFLUENCE import CONFLUENCE

# Set up plotting style
plt.style.use('default')
%matplotlib inline

## 2. Initialize CONFLUENCE
First, let's set up our directories and load the configuration. We'll modify the configuration from Tutorial 3 to use elevation-based discretization.

In [ ]:
# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/work/comphyd_lab/data/CONFLUENCE_data')  # ← User should modify this path

# Verify paths exist
if not CONFLUENCE_CODE_DIR.exists():
    raise FileNotFoundError(f"CONFLUENCE code directory not found: {CONFLUENCE_CODE_DIR}")

if not CONFLUENCE_DATA_DIR.exists():
    print(f"Data directory doesn't exist. Creating: {CONFLUENCE_DATA_DIR}")
    CONFLUENCE_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Load the distributed configuration from Tutorial 3
distributed_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_distributed.yaml'
if not distributed_config_path.exists():
    raise FileNotFoundError("Tutorial 3 must be run first! Distributed config file not found.")

# Read config file
with open(distributed_config_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update core paths
config_dict['CONFLUENCE_CODE_DIR'] = str(CONFLUENCE_CODE_DIR)
config_dict['CONFLUENCE_DATA_DIR'] = str(CONFLUENCE_DATA_DIR)

# Modify for elevation-based discretization
config_dict['DOMAIN_NAME'] = 'Bow_at_Banff_elevation'
config_dict['EXPERIMENT_ID'] = 'elevation_tutorial'
config_dict['DOMAIN_DISCRETIZATION'] = 'elevation'  # Key change!
config_dict['ELEVATION_BAND_SIZE'] = 200  # 200m bands
config_dict['MIN_HRU_SIZE'] = 4  # 4 km² minimum
# Keep SPATIAL_MODE as 'Distributed'

# Save updated config to a temporary file
elevation_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_elevation.yaml'
with open(elevation_config_path, 'w') as f:
    yaml.dump(config_dict, f)

# Initialize CONFLUENCE
confluence = CONFLUENCE(elevation_config_path)

# Display configuration
print("=== Directory Configuration ===")
print(f"Code Directory: {CONFLUENCE_CODE_DIR}")
print(f"Data Directory: {CONFLUENCE_DATA_DIR}")
print("\n=== Key Configuration Settings ===")
print(f"Domain Name: {confluence.config['DOMAIN_NAME']}")
print(f"Pour Point: {confluence.config['POUR_POINT_COORDS']}")
print(f"Discretization Method: {confluence.config['DOMAIN_DISCRETIZATION']}")
print(f"Elevation Band Size: {confluence.config['ELEVATION_BAND_SIZE']} m")
print(f"Minimum HRU Size: {confluence.config['MIN_HRU_SIZE']} km²")
print(f"Spatial Mode: {confluence.config['SPATIAL_MODE']}")
print(f"Model: {confluence.config['HYDROLOGICAL_MODEL']}")
print(f"Simulation Period: {confluence.config['EXPERIMENT_TIME_START']} to {confluence.config['EXPERIMENT_TIME_END']}")

## 3. Project Setup - Organizing the Modeling Workflow

First, we'll establish a well-organized project structure, similar to what we did in Tutorial 3.

In [ ]:
# Step 1: Project Initialization
print("=== Step 1: Project Initialization ===")

# Setup project
project_dir = confluence.managers['project'].setup_project()

# List created directories
print("\nCreated directories:")
for item in sorted(project_dir.iterdir()):
    if item.is_dir():
        print(f"  📁 {item.name}")

## 5. Copy Domain Data from Tutorial 3

We'll reuse the existing domain and GRUs from Tutorial 3 (distributed model) instead of starting from scratch.

In [ ]:
# Source domain from Tutorial 3
source_domain = CONFLUENCE_DATA_DIR / 'domain_Bow_at_Banff_distributed'

# Check if Tutorial 3 domain exists
if not source_domain.exists():
    raise FileNotFoundError("Tutorial 3 domain not found! Please run Tutorial 3 first.")

# Copy necessary directories from Tutorial 3
print("=== Step 2: Copying Domain Data from Tutorial 3 ===")
dirs_to_copy = ['shapefiles', 'attributes']

for dir_name in dirs_to_copy:
    source_dir = source_domain / dir_name
    target_dir = project_dir / dir_name
    
    if source_dir.exists():
        print(f"Copying {dir_name} from Tutorial 3...")
        # Create target directory if it doesn't exist
        target_dir.mkdir(parents=True, exist_ok=True)
        
        # Copy files
        for src_file in source_dir.glob('**/*'):
            if src_file.is_file():
                # Calculate relative path
                rel_path = src_file.relative_to(source_dir)
                dest_file = target_dir / rel_path
                
                # Create parent directories if needed
                dest_file.parent.mkdir(parents=True, exist_ok=True)
                
                # Copy the file, replacing 'distributed' with 'elevation' in filename
                shutil.copy2(src_file, dest_file)
                if 'distributed' in dest_file.name:
                    new_name = dest_file.parent / dest_file.name.replace('distributed', 'elevation')
                    dest_file.rename(new_name)
    else:
        print(f"Warning: {source_dir} not found in Tutorial 3 domain.")

print("\n✓ Domain data copied from Tutorial 3")

## 6. Verify GRU Boundaries

Let's check the existing GRU boundaries that we'll use as a basis for our elevation-based HRUs.

In [ ]:
# Check river basins (GRUs) and network
basin_path = project_dir / 'shapefiles' / 'river_basins'
network_path = project_dir / 'shapefiles' / 'river_network'

basin_files = list(basin_path.glob('*.shp'))
network_files = list(network_path.glob('*.shp'))

if basin_files and network_files:
    # Load data
    basins = gpd.read_file(basin_files[0])
    rivers = gpd.read_file(network_files[0])
    
    # Create visualization
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot basins (GRUs)
    basins.plot(ax=ax, column='GRU_ID', cmap='viridis', 
               alpha=0.7, edgecolor='black', linewidth=1)
    
    # Plot river network
    rivers.plot(ax=ax, color='blue', linewidth=1.5)
    
    # Format plot
    ax.set_title(f'GRU Boundaries for Elevation-Based Discretization\n({len(basins)} Sub-basins)', 
                fontsize=14, fontweight='bold')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    # Add colorbar for GRU IDs
    sm = plt.cm.ScalarMappable(cmap='viridis', 
                             norm=plt.Normalize(vmin=basins['GRU_ID'].min(), 
                                               vmax=basins['GRU_ID'].max()))
    sm._A = []
    cbar = fig.colorbar(sm, ax=ax, shrink=0.8)
    cbar.set_label('GRU ID', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Number of GRUs: {len(basins)}")
else:
    print("GRU files not found. Make sure Tutorial 3 was completed successfully.")

## 7. Create Elevation-Based HRUs

Now we'll apply elevation-based discretization to create HRUs within each GRU.

In [ ]:
# Step 3: Create Elevation-Based HRUs
print("=== Step 3: Creating Elevation-Based HRUs ===")
print(f"Discretization Method: {confluence.config['DOMAIN_DISCRETIZATION']}")
print(f"Elevation Band Size: {confluence.config['ELEVATION_BAND_SIZE']} m")
print(f"Minimum HRU Size: {confluence.config['MIN_HRU_SIZE']} km²")

# Apply discretization
print("\nApplying elevation-based discretization...")
hru_path = confluence.managers['domain'].discretize_domain()

# Load and analyze the resulting HRU shapefile
catchment_path = project_dir / 'shapefiles' / 'catchment'
if catchment_path.exists():
    hru_files = list(catchment_path.glob('*.shp'))
    if hru_files:
        hru_gdf = gpd.read_file(hru_files[0])
        
        # Store the GRU GeoDataFrame for later use
        gru_gdf = gpd.read_file(basin_files[0])
        
        print(f"\n✓ Created elevation-based HRUs")
        print(f"Number of HRUs: {len(hru_gdf)}")
        print(f"Number of GRUs: {hru_gdf['GRU_ID'].nunique()}")
        
        # Calculate HRUs per GRU
        hru_counts = hru_gdf.groupby('GRU_ID').size()
        avg_hrus_per_gru = hru_counts.mean()
        print(f"Average HRUs per GRU: {avg_hrus_per_gru:.1f}")
        
        # Show the first few GRUs with their HRU counts
        print("\nHRUs per GRU (first 10):")
        for gru_id, count in hru_counts.head(10).items():
            print(f"  GRU {gru_id}: {count} HRUs")
else:
    print("Failed to create elevation-based HRUs.")

## 8. Visualize Elevation-Based HRUs

In [ ]:
# Visualize the HRUs
if 'hru_gdf' in locals() and len(hru_gdf) > 0:
    # Check if mean elevation is available in the data
    has_elevation = 'mean_elev' in hru_gdf.columns
    
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Plot HRUs colored by elevation if available, otherwise by HRU ID
    if has_elevation:
        hru_gdf.plot(ax=ax, column='mean_elev', cmap='terrain', 
                   edgecolor='gray', linewidth=0.5, alpha=0.7,
                   legend=True, legend_kwds={'label': 'Mean Elevation (m)'})
    else:
        hru_gdf.plot(ax=ax, column='HRU_ID', cmap='viridis', 
                   edgecolor='gray', linewidth=0.5, alpha=0.7)
    
    # Overlay GRU boundaries
    gru_gdf.boundary.plot(ax=ax, color='red', linewidth=1)
    
    # Add river network for context
    if 'rivers' in locals():
        rivers.plot(ax=ax, color='blue', linewidth=1)
    
    # Add title and labels
    ax.set_title(f'Elevation-Based HRUs\n{len(hru_gdf)} HRUs in {hru_gdf["GRU_ID"].nunique()} GRUs',
                fontsize=16, fontweight='bold')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    plt.tight_layout()
    plt.show()
else:
    print("No HRUs available to visualize.")

## 9. Complete the Modeling Workflow

Now we'll complete the model setup and run the simulation using our elevation-based HRUs.

In [ ]:
# Step 4: Process Observed Data
print("=== Step 4: Processing Observed Data ===")

# Check if we can reuse observed data from previous tutorials
obs_dir = CONFLUENCE_DATA_DIR / 'domain_Bow_at_Banff_distributed' / 'observations' / 'streamflow' / 'preprocessed'
if obs_dir.exists() and list(obs_dir.glob('*.csv')):
    print("Reusing observed data from Tutorial 3...")
    
    # Copy observed data
    target_dir = project_dir / 'observations' / 'streamflow' / 'preprocessed'
    target_dir.mkdir(parents=True, exist_ok=True)
    
    for src_file in obs_dir.glob('*.csv'):
        dest_file = target_dir / src_file.name.replace('distributed', 'elevation')
        shutil.copy2(src_file, dest_file)
        print(f"Copied {src_file.name} to {dest_file.name}")
else:
    print("Processing observed streamflow data from scratch...")
    confluence.managers['data'].process_observed_data()

print("\n✓ Observed data processing completed")

In [ ]:
# Step 5: Acquire and Process Forcing Data
print("=== Step 5: Acquiring and Processing Forcing Data ===")

# Check if we can reuse forcing data from previous tutorials
forcing_dir = CONFLUENCE_DATA_DIR / 'domain_Bow_at_Banff_distributed' / 'forcing'
if forcing_dir.exists():
    print("Reusing forcing data from Tutorial 3...")
    
    # Copy forcing data
    target_dir = project_dir / 'forcing'
    target_dir.mkdir(parents=True, exist_ok=True)
    
    # Copy all subdirectories
    for src_dir in forcing_dir.iterdir():
        if src_dir.is_dir():
            dest_dir = target_dir / src_dir.name
            if not dest_dir.exists():
                shutil.copytree(src_dir, dest_dir)
            print(f"Copied {src_dir.name} forcing data")
    
    # Run model-agnostic preprocessing to handle the new HRU structure
    print("\nRunning model-agnostic preprocessing for new HRU structure...")
    confluence.managers['data'].run_model_agnostic_preprocessing()
else:
    print("Acquiring forcing data from scratch...")
    confluence.managers['data'].acquire_forcings()
    print("\nRunning model-agnostic preprocessing...")
    confluence.managers['data'].run_model_agnostic_preprocessing()

print("\n✓ Forcing data processing completed")

In [ ]:
# Step 6: Prepare Model-Specific Files
print("=== Step 6: Preparing Model-Specific Files ===")

print(f"Preparing {confluence.config['HYDROLOGICAL_MODEL']} input files...")
print(f"This step will generate configuration files for {len(hru_gdf)} HRUs across {hru_gdf['GRU_ID'].nunique()} GRUs")

# Preprocess models
confluence.managers['model'].preprocess_models()

print("\n✓ Model-specific preprocessing completed")

In [ ]:
# Step 7: Run the Model
print("=== Step 7: Running the Model ===")

# Run the model
print(f"Running {confluence.config['HYDROLOGICAL_MODEL']} with elevation-based HRUs...")
print("Note: This may take some time depending on the number of HRUs.")
print(f"Number of HRUs: {len(hru_gdf)}")

# You can comment this out if you want to skip the actual model run
confluence.managers['model'].run_models()